In [1]:
# Test reconstruction performance

In [27]:
import os
import random
import pickle
import warnings
import numpy as np
import pandas as pd

warnings.filterwarnings('ignore')

from keras.optimizers import Adam
from models import AE
# from models import USAD
from models import USAD_keras
from utils import *

def seed_everything(seed=42):
    random.seed(seed)
    np.random.seed(seed)
    os.environ["PYTHONHASHSEED"] = str(seed)
    tf.random.set_seed(seed)
seed_everything(42)

# AutoEncoder(AE)
- LSTM-AE
- GRU-AE
- TCN-AE

In [3]:
# make dataset
df = pd.read_csv('./datasets/JNU_elec.csv').set_index('timestamp')
seq_data = create_seq(df, seq_len=24)

# split(train | val | test)
train = seq_data[:(365 * 24 * 3)]
val = seq_data[(365 * 24 * 3):(365 * 24 * 4)]
test = seq_data[(365 * 24 * 4):]

print(f'Train: {train.shape}')
print(f'Val: {val.shape}')
print(f'Test: {test.shape}')

Train: (26280, 24, 1)
Val: (8760, 24, 1)
Test: (3601, 24, 1)


In [4]:
# Min-Max norm
min_val, max_val = np.min(train), np.max(train)
train_norm = minmax(train, min_val, max_val)
val_norm = minmax(val, min_val, max_val)
test_norm = minmax(test, min_val, max_val)

# make tf dataset
train_dataset = make_dataset(train_norm, train_norm)
val_dataset = make_dataset(val_norm, val_norm)
test_dataset = make_dataset(test_norm, test_norm)

In [5]:
# LSTM-AE
lstmae = AE(mode='LSTM')
# GRU-AE
gruae = AE(mode='GRU')
# TCN-AE
tcnae = AE(z_dim=32, hidden_dim=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
lstmae.build(input_shape)
gruae.build(input_shape)
tcnae.build(input_shape)

lstmae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
gruae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')
tcnae.compile(optimizer=Adam(.001), loss='mse', metrics='mae')

lstm_time = TimeHistory()
gru_time = TimeHistory()
tcn_time = TimeHistory()

In [6]:
print(lstmae.summary(), gruae.summary())

Model: "ae"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder (Encoder)           multiple                  16896     
                                                                 
 decoder (Decoder)           multiple                  115457    
                                                                 
Total params: 132,353
Trainable params: 132,353
Non-trainable params: 0
_________________________________________________________________
Model: "ae_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_1 (Encoder)         multiple                  12864     
                                                                 
 decoder_1 (Decoder)         multiple                  91137     
                                                                 
Total params: 104,001
Trainable params: 104,

In [7]:
print(tcnae.summary())

Model: "ae_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_2 (Encoder)         multiple                  35104     
                                                                 
 decoder_2 (Decoder)         multiple                  16545     
                                                                 
Total params: 51,649
Trainable params: 51,649
Non-trainable params: 0
_________________________________________________________________
None


In [8]:
epochs = 50
batch_size = 64

lstmaehist = lstmae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[lstm_time])
print(f'LSTM-AE train time: {sum(lstm_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 13s 12ms/step - loss: 0.0107 - mae: 0.0703 - val_loss: 0.0055 - val_mae: 0.0504
Epoch 2/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0038 - mae: 0.0420 - val_loss: 0.0045 - val_mae: 0.0440
Epoch 3/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0030 - mae: 0.0366 - val_loss: 0.0040 - val_mae: 0.0418
Epoch 4/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0026 - mae: 0.0340 - val_loss: 0.0039 - val_mae: 0.0405
Epoch 5/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0023 - mae: 0.0323 - val_loss: 0.0035 - val_mae: 0.0389
Epoch 6/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0023 - mae: 0.0317 - val_loss: 0.0034 - val_mae: 0.0382
Epoch 7/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0020 - mae: 0.0300 - val_loss: 0.0029 - val_mae: 0.0344
Epoch 8/50
411/411 [==============================] - 4s 10ms/step -

In [9]:
gruaehist = gruae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[gru_time])
print(f'GRU-AE train time: {sum(gru_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 6s 11ms/step - loss: 0.0226 - mae: 0.1131 - val_loss: 0.0162 - val_mae: 0.0891
Epoch 2/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0059 - mae: 0.0523 - val_loss: 0.0055 - val_mae: 0.0504
Epoch 3/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0040 - mae: 0.0428 - val_loss: 0.0051 - val_mae: 0.0475
Epoch 4/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0035 - mae: 0.0401 - val_loss: 0.0043 - val_mae: 0.0430
Epoch 5/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0027 - mae: 0.0344 - val_loss: 0.0038 - val_mae: 0.0393
Epoch 6/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0024 - mae: 0.0324 - val_loss: 0.0036 - val_mae: 0.0384
Epoch 7/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0023 - mae: 0.0318 - val_loss: 0.0035 - val_mae: 0.0380
Epoch 8/50
411/411 [==============================] - 4s 9ms/step - loss: 0

In [10]:
tcnaehist = tcnae.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcn_time])
print(f'TCN-AE train time: {sum(tcn_time.times):.4f}')

Epoch 1/50


411/411 [==============================] - 7s 9ms/step - loss: 0.0083 - mae: 0.0592 - val_loss: 0.0027 - val_mae: 0.0342
Epoch 2/50
411/411 [==============================] - 4s 9ms/step - loss: 0.0018 - mae: 0.0281 - val_loss: 0.0016 - val_mae: 0.0268
Epoch 3/50
411/411 [==============================] - 4s 10ms/step - loss: 0.0011 - mae: 0.0227 - val_loss: 0.0013 - val_mae: 0.0234
Epoch 4/50
411/411 [==============================] - 4s 10ms/step - loss: 7.3852e-04 - mae: 0.0183 - val_loss: 0.0010 - val_mae: 0.0213
Epoch 5/50
411/411 [==============================] - 4s 9ms/step - loss: 5.6966e-04 - mae: 0.0162 - val_loss: 7.2169e-04 - val_mae: 0.0177
Epoch 6/50
411/411 [==============================] - 4s 9ms/step - loss: 4.5127e-04 - mae: 0.0143 - val_loss: 7.7401e-04 - val_mae: 0.0183
Epoch 7/50
411/411 [==============================] - 4s 9ms/step - loss: 3.6219e-04 - mae: 0.0131 - val_loss: 4.0245e-04 - val_mae: 0.0135
Epoch 8/50
411/411 [==============================] - 4s 

In [11]:
path = './save/JNU/'
check_path(path)
lstmae.save_weights(path + 'LSTMAE.h5')
gruae.save_weights(path + 'GRUAE.h5')
tcnae.save_weights(path + 'TCNAE.h5')

In [32]:
save_hist(lstmaehist, lstm_time, path + 'LSTMAEhist.pkl')
save_hist(gruaehist, gru_time, path + 'GRUAEhist.pkl')
save_hist(tcnaehist, tcn_time, path + 'TCNAEhist.pkl')

# UnSupervisedAnomalyDetection(USAD)
- USAD
- TCN-USAD

In [12]:
# USAD
# usad = USAD(hidden_dims=[128, 64], mode='USAD')
usad = USAD_keras(z_dim=32, hidden_dims=[64, 32], mode='USAD')
# LSTM-USAD
lstmusad = USAD_keras(z_dim=64, hidden_dims=128, mode='LSTM')
# GRU-USAD
gruusad = USAD_keras(z_dim=64, hidden_dims=128, mode='GRU')
# TCN-USAD
tcnusad = USAD_keras(z_dim=32, hidden_dims=32, dilations=[1, 2, 4], mode='TCN')

input_shape = (None, train.shape[1], train.shape[2])
usad.build(input_shape)
lstmusad.build(input_shape)
gruusad.build(input_shape)
tcnusad.build(input_shape)

usad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)
lstmusad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)
gruusad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)
tcnusad.compile(
    g_optimizer=Adam(.001), d_optimizer=Adam(.001)
)

usad_time = TimeHistory()
lstmusad_time = TimeHistory()
gruusad_time = TimeHistory()
tcnusad_time = TimeHistory()

In [13]:
print(usad.summary(), lstmusad.summary())

Model: "usad_keras"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_3 (Encoder)         multiple                  3264      
                                                                 
 decoder_3 (Decoder)         multiple                  3233      
                                                                 
 decoder_4 (Decoder)         multiple                  3233      
                                                                 
Total params: 9,738
Trainable params: 9,730
Non-trainable params: 8
_________________________________________________________________
Model: "usad_keras_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_4 (Encoder)         multiple                  16896     
                                                                 
 decoder_5 (Decoder)         mul

In [14]:
print(gruusad.summary(), tcnusad.summary())

Model: "usad_keras_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_5 (Encoder)         multiple                  12864     
                                                                 
 decoder_7 (Decoder)         multiple                  91137     
                                                                 
 decoder_8 (Decoder)         multiple                  91137     
                                                                 
Total params: 195,146
Trainable params: 195,138
Non-trainable params: 8
_________________________________________________________________
Model: "usad_keras_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 encoder_6 (Encoder)         multiple                  35104     
                                                                 
 decoder_9 (Decoder)      

In [15]:
# usad_times, _ = usad.fit(train_dataset, val_dataset, epochs)
# print(f'USAD train time: {sum(usad_times):.4f}')
usadhist = usad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[usad_time])
print(f'USAD train time: {sum(usad_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 9s 18ms/step - g_loss: 0.0230 - d_loss: 0.0237 - val_g_loss: 1.4571e-04 - val_d_loss: 1.6277e-04
Epoch 2/50
411/411 [==============================] - 7s 17ms/step - g_loss: 1.0021e-04 - d_loss: 1.0890e-04 - val_g_loss: 1.0205e-04 - val_d_loss: 1.1239e-04
Epoch 3/50
411/411 [==============================] - 7s 17ms/step - g_loss: 3.7298e-05 - d_loss: 3.9179e-05 - val_g_loss: 7.8699e-05 - val_d_loss: 8.6917e-05
Epoch 4/50
411/411 [==============================] - 7s 18ms/step - g_loss: 2.4982e-05 - d_loss: 2.6175e-05 - val_g_loss: 7.0264e-05 - val_d_loss: 7.6859e-05
Epoch 5/50
411/411 [==============================] - 7s 18ms/step - g_loss: 2.1664e-05 - d_loss: 2.2697e-05 - val_g_loss: 7.0446e-05 - val_d_loss: 7.7745e-05
Epoch 6/50
411/411 [==============================] - 7s 18ms/step - g_loss: 2.2790e-05 - d_loss: 2.3783e-05 - val_g_loss: 7.9934e-05 - val_d_loss: 8.8993e-05
Epoch 7/50
411/411 [==============================] - 

In [16]:
lstmusadhist = lstmusad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[usad_time])
print(f'LSTM-USAD train time: {sum(lstmusad_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 212s 494ms/step - g_loss: 0.0260 - d_loss: 0.0262 - val_g_loss: 0.0310 - val_d_loss: 0.0312
Epoch 2/50
411/411 [==============================] - 199s 485ms/step - g_loss: 0.0227 - d_loss: 0.0229 - val_g_loss: 0.0305 - val_d_loss: 0.0307
Epoch 3/50
411/411 [==============================] - 203s 494ms/step - g_loss: 0.0229 - d_loss: 0.0231 - val_g_loss: 0.0302 - val_d_loss: 0.0300
Epoch 4/50
411/411 [==============================] - 203s 493ms/step - g_loss: 0.0231 - d_loss: 0.0234 - val_g_loss: 0.0298 - val_d_loss: 0.0295
Epoch 5/50
411/411 [==============================] - 203s 495ms/step - g_loss: 0.0195 - d_loss: 0.0196 - val_g_loss: 0.0273 - val_d_loss: 0.0273
Epoch 6/50
411/411 [==============================] - 203s 493ms/step - g_loss: 0.0091 - d_loss: 0.0095 - val_g_loss: 0.0242 - val_d_loss: 0.0242
Epoch 7/50
411/411 [==============================] - 203s 494ms/step - g_loss: 0.0042 - d_loss: 0.0042 - val_g_loss: 0.0217

In [17]:
gruusadhist = gruusad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[usad_time])
print(f'GRU-USAD train time: {sum(usad_time.times):.4f}')

Epoch 1/50
411/411 [==============================] - 245s 573ms/step - g_loss: 0.0262 - d_loss: 0.0264 - val_g_loss: 0.0285 - val_d_loss: 0.0294
Epoch 2/50
411/411 [==============================] - 235s 571ms/step - g_loss: 0.0181 - d_loss: 0.0182 - val_g_loss: 0.0245 - val_d_loss: 0.0249
Epoch 3/50
411/411 [==============================] - 238s 579ms/step - g_loss: 0.0117 - d_loss: 0.0116 - val_g_loss: 0.0187 - val_d_loss: 0.0191
Epoch 4/50
411/411 [==============================] - 237s 576ms/step - g_loss: 0.0040 - d_loss: 0.0040 - val_g_loss: 0.0157 - val_d_loss: 0.0160
Epoch 5/50
411/411 [==============================] - 236s 574ms/step - g_loss: 0.0037 - d_loss: 0.0038 - val_g_loss: 0.0138 - val_d_loss: 0.0140
Epoch 6/50
411/411 [==============================] - 235s 573ms/step - g_loss: 0.0036 - d_loss: 0.0036 - val_g_loss: 0.0126 - val_d_loss: 0.0127
Epoch 7/50
411/411 [==============================] - 236s 575ms/step - g_loss: 0.0034 - d_loss: 0.0034 - val_g_loss: 0.0115

In [18]:
# tcnusad_times, _ = tcnusad.fit(train_dataset, val_dataset, epochs)
# print(f'TCN-USAD train time: {sum(tcnusad_times):.4f}')
tcnusadhist = tcnusad.fit(train_dataset, epochs=epochs, batch_size=batch_size, validation_data=(val_dataset), callbacks=[tcnusad_time])
print(f'TCN-USAD train time: {sum(tcnusad_time.times):.4f}')

Epoch 1/50


411/411 [==============================] - 11s 20ms/step - g_loss: 0.0224 - d_loss: 0.0242 - val_g_loss: 0.0069 - val_d_loss: 0.0073
Epoch 2/50
411/411 [==============================] - 8s 20ms/step - g_loss: 0.0044 - d_loss: 0.0045 - val_g_loss: 0.0056 - val_d_loss: 0.0060
Epoch 3/50
411/411 [==============================] - 8s 20ms/step - g_loss: 0.0030 - d_loss: 0.0032 - val_g_loss: 0.0049 - val_d_loss: 0.0052
Epoch 4/50
411/411 [==============================] - 8s 19ms/step - g_loss: 0.0023 - d_loss: 0.0024 - val_g_loss: 0.0046 - val_d_loss: 0.0049
Epoch 5/50
411/411 [==============================] - 8s 19ms/step - g_loss: 0.0019 - d_loss: 0.0019 - val_g_loss: 0.0043 - val_d_loss: 0.0045
Epoch 6/50
411/411 [==============================] - 8s 19ms/step - g_loss: 0.0014 - d_loss: 0.0014 - val_g_loss: 0.0038 - val_d_loss: 0.0041
Epoch 7/50
411/411 [==============================] - 8s 19ms/step - g_loss: 0.0013 - d_loss: 0.0013 - val_g_loss: 0.0035 - val_d_loss: 0.0037
Epoch 8/5

In [19]:
path = './save/JNU/'
check_path(path)
usad.save_weights(path + 'USAD.h5')
lstmusad.save_weights(path + 'LSTMUSAD.h5')
gruusad.save_weights(path + 'GRUUSAD.h5')
tcnusad.save_weights(path + 'TCNUSAD.h5')

In [31]:
save_hist(usadhist, usad_time, path + 'USADhist.pkl')
save_hist(lstmusadhist, lstmusad_time, path + 'LSTMUSADhist.pkl')
save_hist(gruusadhist, gruusad_time, path + 'GRUUSADhist.pkl')
save_hist(tcnusadhist, tcnusad_time, path + 'TCNUSADhist.pkl')

In [20]:
# To-Do
# 00. make TCN-USAD(Done)
# 00. train USAD with keras fit
# 01. prepare dataset(LEAD 1.0, EPIC, aihub)(ing)
# 02. test and make a Smart Media Journal Paper

# # 모델 가중치 학습 가능 여부 확인 코드
# for weight in autoencoder.encoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# for weight in autoencoder.decoder.trainable_weights:
#     print(f"Weight name: {weight.name}, Trainable: {weight.trainable}")
# autoencoder.encoder.trainable_weights
# autoencoder.decoder.trainable_weights

In [21]:
# # save model weights
# path = './save/JNU/'
# check_path(path)
# lstmae.save(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')
# gruae.save(e_path=path+'GRU_Encoder.h5', d_path=path+'GRU_Decoder.h5')
# tcnae_s.save(e_path=path+'TCN_Encoder_s.h5', d_path=path+'TCN_Decoder_s.h5')
# tcnae_l.save(e_path=path+'TCN_Encoder_l.h5', d_path=path+'TCN_Decoder_l.h5')

In [22]:
# path = './save/JNU/'
# lstmae.load(e_path=path+'LSTM_Encoder.h5', d_path=path+'LSTM_Decoder.h5')